# Kaggle: Predict Loan Payback — Data Cleaning

**Notebook:** `02_data_cleaning.ipynb`
**Author:** Brice Nelson
**Organization:** Kaggle Series | Brice Machine Learning Projects
**Date Created:** November 2, 2025
**Last Updated:** November 15, 2025

---

## 🧭 Purpose

This notebook performs **data cleaning and validation** for the Kaggle *Predict Loan Payback* dataset.
The focus is on ensuring the **train** and **test** datasets are structurally aligned and free of inconsistencies prior to feature engineering and model training.

### **Objectives**
1. Load and inspect both train and test datasets.
2. Validate schema consistency (columns, dtypes, and shapes).
3. Identify and address any missing, duplicated, or outlier values.
4. Standardize formatting across categorical and numeric fields.

---

## 🔍 Dataset Comparison Overview

Before applying cleaning operations, it is essential to verify that both datasets share compatible structures.
The following checks confirm that column names, data types, and row counts align as expected.


## Import Libraries

In [17]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler



In [2]:
# ============================================================
# 🔍 Train vs Test Structure Validation
# ============================================================

# Load datasets
train_path = "../data/raw/train.csv"
test_path = "../data/raw/test.csv"

loan_train_df = pd.read_csv(train_path)
loan_test_df = pd.read_csv(test_path)

# Basic shape and info comparison
print(f"Train Shape: {loan_train_df.shape}")
print(f"Test Shape:  {loan_test_df.shape}\n")

print("Train Columns:", loan_train_df.columns.tolist())
print("Test Columns:", loan_test_df.columns.tolist())

# Define known target column(s)
target_cols = {"loan_paid_back"}

# Check for any column mismatches
train_only_cols = set(loan_train_df.columns) - set(loan_test_df.columns) - target_cols
test_only_cols = set(loan_test_df.columns) - set(loan_train_df.columns)

if train_only_cols or test_only_cols:
    print("\n⚠️ Column mismatches detected:")
    if train_only_cols:
        print("Columns only in train (excluding target):", train_only_cols)
    if test_only_cols:
        print("Columns only in test:", test_only_cols)
else:
    print("\n✅ Train and test datasets have matching columns (except for target variable).")

# Quick dtype consistency check (only for common columns)
common_cols = loan_train_df.columns.intersection(loan_test_df.columns)
dtype_diff = loan_train_df[common_cols].dtypes != loan_test_df[common_cols].dtypes

if dtype_diff.any():
    print("\n⚠️ Data type mismatches found in the following columns:")
    print(loan_train_df[common_cols].dtypes[dtype_diff])
else:
    print("\n✅ Data types are consistent across train and test datasets.")



Train Shape: (593994, 13)
Test Shape:  (254569, 12)

Train Columns: ['id', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade', 'loan_paid_back']
Test Columns: ['id', 'annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate', 'gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']

✅ Train and test datasets have matching columns (except for target variable).

✅ Data types are consistent across train and test datasets.


In [3]:
loan_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  grade_subgrade        593994 non-null  object 
 12  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(2), object(6)
memory usage: 58.9+ MB


# Categorical Columns

| Column           | Type of Encoding | # of Categories          | 1     | 2        | 3             | 4         | 5        | 6     | 7     | 8     |
|------------------|------------------|-----------------|-------|----------|---------------|-----------|----------|-------|-------|-------|
| gender           | One Hot Encoding | 3               | male                     | female   | other         |           |          |
| marital_status   | One Hot Encoding | 4               | single                   | married  | divorced      | widowed   |          |
| education_level  | One Hot Encoding | 5               | high_school              | bachelor  | master  | phd       | other    |
| employment_status| One Hot Encoding | 5               | employed                 | unemployed| self_employed | retired   | student  |
| loan_purpose     | One Hot Encoding | 8               | home                     | debt_consolidation | car           | education | business | medical | vacation | other |



## Confirm Missing Data

In [4]:
missing_data = pd.DataFrame({'Total Missing':loan_train_df.isnull().sum()})
if missing_data['Total Missing'].any():
    print(missing_data)
else:
    print('Data contains no null values')

Data contains no null values


## One Hot Encoding vs. Ordinal Encoding
* Ordinal Encoding ranks, One Hot Encoding does not
* One Hot Encoding is more interpretable
* Ordinal Encoding cannot contain noise in the data (ie: other has no ranking in education_level and cannot interpet it)
* One Hot Encoding can contain noise in the data (ie: other has a ranking in employment_status)
* Data needs to be explored before makine the decision to use OHE vs OE

In [5]:
# Explore `other` in `education_level`.  Can `other` be dropped
loan_train_df['education_level'].value_counts(normalize=True)

education_level
Bachelor's     0.470722
High School    0.309081
Master's       0.156731
Other          0.044911
PhD            0.018556
Name: proportion, dtype: float64

### One Hot Encoding vs Ordinal Encoding Decision
When preparing categorical features, a natural question arises:
Should the variable be encoded as ordinal (ranked) or one-hot (unordered)?

For categories that naturally follow a progression (e.g., high_school < bachelor < master < phd), ordinal encoding can make sense — but only if all categories fit cleanly into that ranking.
Before deciding, it’s important to look at the distribution of categories, especially any catch-all or ambiguous labels such as `Other`.

| % of dataset | Typical action          | Rationale                          |
| ------------ | ----------------------- | ---------------------------------- |
| < 0.1%       | Drop freely             | Random noise, typo bucket          |
| < 0.5%       | Drop if needed          | Still small enough                 |
| 0.5%–1%      | Be cautious             | Dropping can distort distributions |
| 1%–2%        | Rarely drop             | That’s meaningful population       |
| 4%–5%        | **Never drop**          | This is a real category            |
| >5%          | Treat as major category | Keep, understand, encode properly  |

**Applying This to education_level**

* The `Other` category in the training dataset accounts for 4.49% of all observations — roughly 1 in 20 data points.
* A category this large cannot be discarded without introducing bias or removing meaningful structure from the data.
* Because education_level contains a catch-all `Other` bucket that does not fit cleanly into an ordered hierarchy, ordinal encoding would create an artificial ranking and inject false relationships.

**Final Decision**

* Use One-Hot Encoding for education_level.
* This preserves all information, avoids incorrect ordinal assumptions, and reflects real-world feature engineering practice.

## Ordinal Encoding
The dataset includes a categorical variable called grade_subgrade, which represents a lender’s internal credit risk tier assigned to each loan applicant. This system is inherently ordinal:

* The letter grade (A–G) indicates the overall credit quality, where A is the lowest-risk tier and G is the highest-risk tier.
* The subgrade (1–5) provides a finer ranking within each grade, e.g., A1 is better than A5, and B1 is better than B5.

Because both components follow a strict, meaningful order, they should not be one-hot encoded. Instead:

1. Split the combined string (grade_subgrade) into two columns:
    * `grade`: first character (A–G)
    * `subgrade`: numeric rating within that grade (1–5)

2. Encode each component using ordinal encoding, preserving the lender’s risk hierarchy:
    * `grade`: A=1, B=2, … G=7
    * `subgrade`: already numeric, use directly as an ordered variable

This approach maintains the underlying credit-risk structure and avoids losing valuable predictive signal.

In [6]:
# --- Ordinal Encoding for grade_subgrade ---
# This version is safe to run multiple times without KeyErrors.

# Split only if column exists
if 'grade_subgrade' in loan_train_df.columns:

    # Extract grade (Letter) and subgrade (Digit)
    loan_train_df['grade'] = loan_train_df['grade_subgrade'].str[0]
    loan_train_df['subgrade'] = loan_train_df['grade_subgrade'].str[1].astype(int)

    loan_test_df['grade'] = loan_test_df['grade_subgrade'].str[0]
    loan_test_df['subgrade'] = loan_test_df['grade_subgrade'].str[1].astype(int)

    # Drop the original combined column
    loan_train_df = loan_train_df.drop(columns=['grade_subgrade'])
    loan_test_df = loan_test_df.drop(columns=['grade_subgrade'])

# Ordinal encoding mapping
grade_mapping = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7}

# Map grade values (safe even if it already exists)
loan_train_df['grade'] = loan_train_df['grade'].map(grade_mapping)
loan_test_df['grade'] = loan_test_df['grade'].map(grade_mapping)

# sanity print
print(loan_train_df
      [['grade', 'subgrade']].head())


   grade  subgrade
0      3         3
1      4         3
2      3         5
3      6         1
4      4         1


In [7]:
# --- One-Hot Encoding for categorical columns (SAFE VERSION) ---

# Define categorical columns
cat_cols = [
    "gender",
    "marital_status",
    "education_level",
    "employment_status",
    "loan_purpose"
]

# Only attempt encoding if ALL columns exist
missing_columns = [col for col in cat_cols if col not in loan_train_df.columns]
if missing_columns:
    raise KeyError(f"The following categorical columns are missing and cannot be encoded: {missing_columns}")

# Combine for fitting (avoids mismatched categories)
combined_cat_data = pd.concat([loan_train_df[cat_cols], loan_test_df[cat_cols]], axis=0)

# Create encoder
ohe = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

# Fit on combined
ohe.fit(combined_cat_data)

# Transform train/test
train_ohe = ohe.transform(loan_train_df[cat_cols])
test_ohe = ohe.transform(loan_test_df[cat_cols])

# OHE feature names
ohe_cols = ohe.get_feature_names_out(cat_cols)

# Convert to DataFrames (safe — does not modify original)
loan_train_ohe_df = pd.DataFrame(train_ohe, columns=ohe_cols, index=loan_train_df.index)
loan_test_ohe_df = pd.DataFrame(test_ohe, columns=ohe_cols, index=loan_test_df.index)

print("OHE encoding complete. Encoded DataFrame shapes:")
print("Train OHE:", loan_train_ohe_df.shape)
print("Test OHE:", loan_test_ohe_df.shape)




OHE encoding complete. Encoded DataFrame shapes:
Train OHE: (593994, 25)
Test OHE: (254569, 25)


In [10]:
# --- Final merge for encoded features (SAFE VERSION) ---

# Make copies so originals are never modified
train_tmp = loan_train_df.copy()
test_tmp = loan_test_df.copy()

# Drop original categorical columns ONLY if they exist
train_tmp = train_tmp.drop(columns=[col for col in cat_cols if col in train_tmp.columns])
test_tmp = test_tmp.drop(columns=[col for col in cat_cols if col in test_tmp.columns])

# Drop grade_subgrade if still present (safe)
for col in ['grade_subgrade']:
    if col in train_tmp.columns:
        train_tmp = train_tmp.drop(columns=[col])
    if col in test_tmp.columns:
        test_tmp = test_tmp.drop(columns=[col])

# Add OHE features (does not overlap with existing columns)
loan_train_encoded = pd.concat([train_tmp, loan_train_ohe_df], axis=1)
loan_test_encoded = pd.concat([test_tmp, loan_test_ohe_df], axis=1)

print("Train encoded shape:", loan_train_encoded.shape)
print("Test encoded shape:", loan_test_encoded.shape)



Train encoded shape: (593994, 34)
Test encoded shape: (254569, 33)


In [11]:
# --- Sanity check: Ensure matching features between train and test ---

train_cols = set(loan_train_encoded.columns) - {"loan_paid_back"}  # remove target
test_cols = set(loan_test_encoded.columns)

if train_cols != test_cols:
    extra_in_train = train_cols - test_cols
    extra_in_test = test_cols - train_cols

    raise AssertionError(
        f"ERROR: Train and Test columns do NOT match!\n"
        f"Columns only in TRAIN: {extra_in_train}\n"
        f"Columns only in TEST : {extra_in_test}"
    )

print("Column alignment check passed — train/test feature sets match.")



Column alignment check passed — train/test feature sets match.


In [12]:
loan_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    593994 non-null  int64  
 1   annual_income         593994 non-null  float64
 2   debt_to_income_ratio  593994 non-null  float64
 3   credit_score          593994 non-null  int64  
 4   loan_amount           593994 non-null  float64
 5   interest_rate         593994 non-null  float64
 6   gender                593994 non-null  object 
 7   marital_status        593994 non-null  object 
 8   education_level       593994 non-null  object 
 9   employment_status     593994 non-null  object 
 10  loan_purpose          593994 non-null  object 
 11  loan_paid_back        593994 non-null  float64
 12  grade                 593994 non-null  int64  
 13  subgrade              593994 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 63.

In [13]:
loan_train_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 34 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   id                               593994 non-null  int64  
 1   annual_income                    593994 non-null  float64
 2   debt_to_income_ratio             593994 non-null  float64
 3   credit_score                     593994 non-null  int64  
 4   loan_amount                      593994 non-null  float64
 5   interest_rate                    593994 non-null  float64
 6   loan_paid_back                   593994 non-null  float64
 7   grade                            593994 non-null  int64  
 8   subgrade                         593994 non-null  int64  
 9   gender_Female                    593994 non-null  float64
 10  gender_Male                      593994 non-null  float64
 11  gender_Other                     593994 non-null  float64
 12  ma

In [14]:
loan_train_encoded.head()


,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,loan_paid_back,grade,subgrade,gender_Female,...,employment_status_Student,employment_status_Unemployed,loan_purpose_Business,loan_purpose_Car,loan_purpose_Debt consolidation,loan_purpose_Education,loan_purpose_Home,loan_purpose_Medical,loan_purpose_Other,loan_purpose_Vacation
0,0,29367.99,0.084,736,2528.42,13.67,1.0,3,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,22108.02,0.166,636,4593.10,12.92,0.0,4,3,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,49566.20,0.097,694,17005.15,9.76,1.0,3,5,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,46858.25,0.065,533,4682.48,16.10,1.0,6,1,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,25496.70,0.053,665,12184.43,10.21,1.0,4,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# 🧮 Scaling Numeric Features

## Why Scaling Is Necessary
Many machine learning algorithms are sensitive to differences in feature scales. In our dataset, numeric variables span very different ranges:

- **loan_amount** ranges from a few thousand dollars to tens of thousands
- **interest_rate** ranges from roughly 5–20%
- **credit_score** ranges from 300–850
- **debt_to_income_ratio** ranges from 0.0–1.0

Without scaling, models such as:

- Logistic Regression
- Linear/ElasticNet models
- Support Vector Machines
- Neural Networks
- K-Nearest Neighbors

may unintentionally give more weight to large-magnitude features simply because of their scale, not because they are more predictive.

Tree-based models (Random Forest, XGBoost, LightGBM, CatBoost) are scale-invariant, but scaling does **not hurt** them and keeps a unified preprocessing pipeline across all model types.

---

## Features Selected for Scaling
We scale the following numeric features:

- `annual_income`
- `debt_to_income_ratio`
- `credit_score`
- `loan_amount`
- `interest_rate`
- `grade` (ordinal)
- `subgrade` (ordinal)

One-Hot Encoded categorical variables are **not scaled** because:

- They are already in a consistent 0/1 range
- Scaling them provides no modeling benefit
- It can distort their interpretation

---

## How We Scale
We use **StandardScaler**, which transforms each numeric column using:

$$
z = \frac{x - \mu}{\sigma}
$$

This transformation ensures:

- Mean = 0
- Standard deviation = 1
- Faster gradient-based optimization
- More stable coefficients for linear models
- Better numerical conditioning for SVM/NN models

---

## Avoiding Data Leakage
To preserve model integrity, the scaler is:

1. **Fit on the training data only**, then
2. **Applied to both train and test sets**

This prevents the model from “seeing” test-set information during preprocessing.

---

## Output DataFrames
After scaling, we produce:

- `loan_train_scaled` → fully encoded + scaled training data
- `loan_test_scaled` → fully encoded + scaled test data

These DataFrames:

- Contain only numeric features
- Have no raw object/string columns
- Include all OHE and ordinal encodings
- Are standardized and ready for feature engineering + modeling


In [16]:
# --- Scale Numeric Features (SAFE IMPLEMENTATION) ---

# Columns to scale
numeric_cols = [
    "annual_income",
    "debt_to_income_ratio",
    "credit_score",
    "loan_amount",
    "interest_rate",
    "grade",
    "subgrade",
]

# Create scaler
scaler = StandardScaler()

# Create copies so original encoded data stays clean
loan_train_scaled = loan_train_encoded.copy()
loan_test_scaled = loan_test_encoded.copy()

# Ensure columns exist before scaling
missing_cols = [col for col in numeric_cols if col not in loan_train_scaled.columns]
if missing_cols:
    raise KeyError(f"Cannot scale. Missing columns: {missing_cols}")

# Fit on TRAIN, apply to TRAIN + TEST (avoid data leakage)
loan_train_scaled[numeric_cols] = scaler.fit_transform(
    loan_train_encoded[numeric_cols]
)
loan_test_scaled[numeric_cols] = scaler.transform(
    loan_test_encoded[numeric_cols]
)

print("Scaling complete.")
print("Train scaled shape:", loan_train_scaled.shape)
print("Test scaled shape:", loan_test_scaled.shape)


Scaling complete.
Train scaled shape: (593994, 34)
Test scaled shape: (254569, 33)


In [18]:
# --- Export processed datasets ---

# Create processed directory if it doesn't exist
processed_path = "../data/processed"
os.makedirs(processed_path, exist_ok=True)

# File paths
train_path = os.path.join(processed_path, "loan_train_scaled.csv")
test_path = os.path.join(processed_path, "loan_test_scaled.csv")

# Export
loan_train_scaled.to_csv(train_path, index=False)
loan_test_scaled.to_csv(test_path, index=False)

print("Export complete:")
print(f" - {train_path}")
print(f" - {test_path}")


Export complete:
 - ../data/processed/loan_train_scaled.csv
 - ../data/processed/loan_test_scaled.csv


# ✅ Summary: Data Cleaning & Preprocessing

This notebook completed all core preprocessing steps required to transform the raw Kaggle loan dataset into a clean, numeric, model-ready form. The following tasks were performed:

---

## 🔹 1. Ordinal Encoding: `grade_subgrade`
The lender-provided credit tier (`grade_subgrade`) was split into:
- `grade` (A–G → 1–7 ordinal scale)
- `subgrade` (1–5 numeric)

This preserves the inherent credit-risk ordering that lenders use when assigning borrower quality.

---

## 🔹 2. One-Hot Encoding for Categorical Variables
The following categorical fields were converted into numeric indicator features using `OneHotEncoder`:

- `gender`
- `marital_status`
- `education_level`
- `employment_status`
- `loan_purpose`

These features do **not** have meaningful ordinal structure, so OHE prevents the model from inferring false ordering.

---

## 🔹 3. Train/Test Alignment
To avoid mismatched columns between train and test:
- OHE was fit on **combined** train + test categories
- A validation check ensured both final matrices contain identical feature sets
- This prevents unseen-category failures during inference

---

## 🔹 4. Scaling Numeric Variables
All numeric features were standardized using **StandardScaler**:

$$
z = \frac{x - \mu}{\sigma}
$$

The following columns were scaled:

- `annual_income`
- `debt_to_income_ratio`
- `credit_score`
- `loan_amount`
- `interest_rate`
- `grade`
- `subgrade`

Scaling improves model stability for linear, SVM, and neural models while keeping tree models unaffected.

---

## 🔹 5. Processed Dataset Export
Final preprocessed matrices were saved to:

